In [ ]:
# | default_exp components.user

Note: 

While writing doc strings, please use the below syntax for linking methods/classes. So that the methods/classes gets highlighted in the browser and clicking on it will take the user to the linked function

    - To link a method from the class same file please use the `method_name` format.
    - To link a method from a different Class (can in a seperate file also) please use `Classname.method_name` format.

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

from typing import *

In [ ]:
# | exporti

import os

import pandas as pd
import qrcode
from fastcore.foundation import patch
from qrcode.image.pil import PilImage

from airt.components.client import Client
from airt.helper import (
    check_and_append_otp_query_param,
    delete_data,
    generate_df,
    get_attributes_from_instances,
    get_data,
    post_data,
    standardize_phone_number,
)
from airt.logger import get_logger, set_level

In [ ]:
import logging
from contextlib import contextmanager
from random import randrange

import pytest

import airt.sanitizer
from airt.constant import (
    SERVICE_PASSWORD,
    SERVICE_SUPER_USER,
    SERVICE_TOKEN,
    SERVICE_USERNAME,
)
from airt.docstring.helpers import run_examples_from_docstring

In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.INFO

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

20

[INFO] __main__: This is an info
[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


In [ ]:
INVALID_UUID_FOR_TESTING = "00000000-0000-0000-0000-000000000000"

In [ ]:
# | export


class User:
    """A class for creating, managing, and updating users on the server.

    The **User** class has two types of methods:

    * Methods for creating and managing users.
    * Method for updating and adding additional security to user accounts.

    Methods such as `create`, `enable`, `disable`, and `ls` can be used to manage user accounts on the server, but access to them requires super user privileges.

    The remaining methods do not require super user privileges and are used to update/additionally secure user accounts.

    In addition to the regular authentication with credentials, the users can enable multi-factor authentication (MFA) and single sign-on (SSO) for their accounts.

    To help protect your account, we recommend that you enable multi-factor authentication (MFA). MFA provides additional security by requiring you to provide unique verification code (OTP) in addition to your regular sign-in credentials when performing critical operations.

    Your account can be configured for MFA in just two easy steps:

    - To begin, you need to enable MFA for your account by calling the `enable_mfa` method, which will generate a QR code. You can then scan the QR code with an authenticator app, such as Google Authenticator and follow the on-device instructions to finish the setup in your smartphone.

    - Finally, activate MFA for your account by calling `activate_mfa` and passing the dynamically generated six-digit verification code from your smartphone's authenticator app.

    Single sign-on (SSO) can be enabled for your account in three simple steps:

    - Enable the SSO for a provider by calling the `enable_sso` method with the SSO provider name and an email address. At the moment, we only support "google" and "github" as SSO providers. We intend to support additional SSO providers in future releases.

    - Before you can start generating new tokens with SSO, you must first authenticate with the SSO provider. Call the get_token with the same SSO provider you have enabled in the step above to generate an SSO authorization URL. Please copy and paste it into your preferred browser and complete the authentication process with the SSO provider.

    - After successfully authenticating with the SSO provider, call the `Client.set_sso_token` method to generate a new token and use it automatically in all future interactions with the airt server.

    Here's an example of using the User class's methods to display the logged-in user's uuid

    Example:
        ```python
        # Importing necessary libraries
        from  airt.client import User, Client

        # Authenticate
        Client.get_token(username="{fill in username}", password="{fill in password}")

        # Display the uuid of logged-in user
        user_details = User.details()
        print(user_details["uuid"])
        ```
    """

    USER_COLS = [
        "uuid",
        "username",
        "email",
        "super_user",
        "is_mfa_active",
        "disabled",
        "created",
        "subscription_type",
        "first_name",
        "last_name",
        "phone_number",
        "is_phone_number_verified",
    ]

    def __init__(
        self,
        uuid: str,
        username: Optional[str] = None,
        first_name: Optional[str] = None,
        last_name: Optional[str] = None,
        email: Optional[str] = None,
        subscription_type: Optional[str] = None,
        super_user: Optional[bool] = None,
        disabled: Optional[str] = None,
        created: Optional[str] = None,
        is_mfa_active: Optional[bool] = None,
        phone_number: Optional[str] = None,
        is_phone_number_verified: Optional[bool] = None,
    ):
        """Constructs a new User instance.

        Args:
            uuid: User uuid.
            username: The username of the user.
            first_name: The first name of the user.
            last_name: The last name of the user.
            email: The email address of the user.
            subscription_type: User subscription type. Currently, the API supports only the following subscription
                types **small**, **medium** and **large**.
            super_user: Flag to indicate the user type.
            disabled: Flag to indicate the status of the user.
            created: User creation date.
            is_mfa_active: Flag to indicate the user's MFA status.
            phone_number: The registered phone number of the user. The phone number should follow the pattern of country
                code followed by your phone number. For example, **440123456789, +440123456789, 00440123456789, +44 0123456789,
                and (+44) 012 345 6789** are all valid formats for registering a UK phone number.
            is_phone_number_verified: Flag to indicate the phone number registration status.
        """

        self.uuid = uuid
        self.username = username
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.subscription_type = subscription_type
        self.super_user = super_user
        self.disabled = disabled
        self.created = created
        self.is_mfa_active = is_mfa_active
        self.phone_number = phone_number
        self.is_phone_number_verified = is_phone_number_verified

    @staticmethod
    def create(
        *,
        username: str,
        first_name: str,
        last_name: str,
        email: str,
        password: str,
        subscription_type: str,
        super_user: bool = False,
        phone_number: Optional[str] = None,
        otp: Optional[str] = None,
    ) -> pd.DataFrame:
        """Create a new user in the server.

        To access this method, you must have super user privileges.

        Args:
            username: The new user's username. The username must be unique or an exception will be thrown.
            first_name: The new user's first name.
            last_name: The new user's last name.
            email: The new user's email. The email must be unique or an exception will be thrown.
            password: The new user's password.
            subscription_type: User subscription type. Currently, the API supports only the following subscription
                types **small**, **medium** and **large**.
            super_user: If set to **True**, then the new user will have super user privilages.
                If **None**, then the default value **False** will be used to create a non-super user.
            phone_number: Phone number to be added to the user account. The phone number should follow the pattern of the country
                code followed by your phone number. For example, **440123456789, +440123456789, 00440123456789, +44 0123456789,
                and (+44) 012 345 6789** are all valid formats for registering a UK phone number.
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
                A pandas DataFrame encapsulating the details of the newly created user.

        Raises:
            ConnectionError: If the server address is invalid or not reachable.
            ValueError: If the OTP is invalid.
            ValueError: If the username or email is already present in the server.

        Below is an example of creating a new user

        Example:
            ```python

            # Importing necessary libraries
            from  airt.client import Client, User

            # Authenticate with super user privileges
            Client.get_token(
                username="{fill in super_user_username}",
                password="{fill in super_user_password}"
            )

            # Details required to create a new user
            username = "{fill in username}"
            first_name = "{fill in first_name}"
            last_name = "{fill in last_name}"
            email = "{fill in email}"
            password = "{fill in password}"
            super_user = "{fill in super_user}"
            subscription_type = "{fill in subscription_type}"

            # Create a new user. To access this method, you must have super user privileges.
            new_user = User.create(
                username=username,
                first_name=first_name,
                last_name=last_name,
                email=email,
                password=password,
                super_user=super_user,
                subscription_type=subscription_type,
            )

            # Display the details of the newly created user
            print(new_user)

            # An exception will be raised if you attempt to create a new user
            # with an already-used username or email address.
            try:
                User.create(
                    username=username,
                    first_name=first_name,
                    last_name=last_name,
                    email=email,
                    password=password,
                    super_user=super_user,
                    subscription_type=subscription_type,
                )
                print("Should not print this, the above line should raise an exception")
                raise RuntimeException()

            except ValueError as e:
                print("Expected to fail, everything is fine")
            ```
        """

        if phone_number is not None:
            phone_number = standardize_phone_number(phone_number)

        req_json = dict(
            username=username,
            first_name=first_name,
            last_name=last_name,
            email=email,
            subscription_type=subscription_type,
            super_user=super_user,
            password=password,
            phone_number=phone_number,
            otp=otp,
        )

        response = Client._post_data(relative_url=f"/user/", json=req_json)

        return pd.DataFrame(response, index=[0])[User.USER_COLS]

    @staticmethod
    def ls(
        offset: int = 0,
        limit: int = 100,
        disabled: bool = False,
    ) -> List["User"]:
        """Return the list of User instances available in the server.

        To access this method, you must have super user privileges.

        Args:
            offset: The number of users to offset at the beginning. If **None**, then the default value **0** will be used.
            limit: The maximum number of users to return from the server. If None, then the default value 100 will be used.
            disabled: If set to **True**, then only the deleted users will be returned. Else, the default value **False** will
                be used to return only the list of active users.

        Returns:
            A list of User instances available in the server.

        Raises:
            ConnectionError: If the server address is invalid or not reachable.

        An example of displaying the details of all active users

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Authenticate with super user privileges
            Client.get_token(
                username="{fill in super_user_username}",
                password="{fill in super_user_password}"
            )

            # Display the details of all active users
            # Set the disabled parameter to True to display the details of inactive users
            ux = User.ls()
            print(User.as_df(ux))
            ```
        """
        users = Client._get_data(
            relative_url=f"/user/?disabled={disabled}&offset={offset}&limit={limit}"
        )

        ux = [
            User(
                uuid=user["uuid"],
                username=user["username"],
                first_name=user["first_name"],
                last_name=user["last_name"],
                email=user["email"],
                subscription_type=user["subscription_type"],
                super_user=user["super_user"],
                disabled=user["disabled"],
                created=user["created"],
                is_mfa_active=user["is_mfa_active"],
                phone_number=user["phone_number"],
                is_phone_number_verified=user["is_phone_number_verified"],
            )
            for user in users
        ]

        return ux

    @staticmethod
    def as_df(ux: List["User"]) -> pd.DataFrame:
        """Return the details of User instances as a pandas dataframe.

        Args:
            ux: List of user instances.

        Returns:
            Details of all the User in a dataframe.

        Raises:
            ConnectionError: If the server address is invalid or not reachable.

        An example of displaying the details of all active users

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Authenticate with super user privileges
            # Only super users can get the list of available users
            Client.get_token(
                username="{fill in super_user_username}",
                password="{fill in super_user_password}"
            )

            # Display the details of all active users
            # Set the disabled parameter to True to display the details of inactive users
            ux = User.ls()
            print(User.as_df(ux))
            ```
        """
        lists = get_attributes_from_instances(ux, User.USER_COLS)  # type: ignore
        return generate_df(lists, User.USER_COLS)

    @staticmethod
    def disable(user: Union[str, List[str]], otp: Optional[str] = None) -> pd.DataFrame:
        """Disable one or more users.

        To access this method, you must have super user privileges.

        Args:
            user: user_uuid/username to disabled. To disable multiple users, please pass the uuids/names as a list.
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
            A pandas DataFrame encapsulating the details of the disabled user.

        Raises:
            ValueError: If the OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        Here's an example to disable a user

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Authenticate with super user privileges
            Client.get_token(
                username="{fill in super_user_username}",
                password="{fill in super_user_password}"
            )

            # Optional Step: For demonstration purpose, create a new user
            username = "{fill in username}"
            User.create(
                username=username,
                first_name="{fill in first_name}",
                last_name="{fill in last_name}",
                email="{fill in email}",
                password="{fill in password}",
                subscription_type="{fill in subscription_type}",
            )

            # Display the details of the user you want to disable using their username/uuid
            print(User.details(username))

            # Disable the user
            # To disable multiple users, pass a list of username/uuid
            User.disable(user=username)

            # Check whether the user has been disabled
            # The disabled flag should be set to True, if the disable was sucessful
            print(User.details(username))
            ```
        """
        _users = user if isinstance(user, list) else [user]
        response_list = []
        for user in _users:
            user_uuid = User.details(user=user)["uuid"]
            url = f"/user/{user_uuid}"
            response = Client._delete_data(
                relative_url=check_and_append_otp_query_param(url, otp)
            )
            response_list.append(response)

        return pd.DataFrame(response_list)[User.USER_COLS]

    @staticmethod
    def enable(user: Union[str, List[str]], otp: Optional[str] = None) -> pd.DataFrame:
        """Enable one or more disabled users.

        To access this method, you must have super user privileges.

        Args:
            user: user_uuid/username to enable. To enable multiple users, please pass the uuids/names as a list.
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
            A pandas DataFrame encapsulating the details of the enabled user.

        Raises:
            ValueError: If the OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        Here's an example to enable a user

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Authenticate with super user privileges
            Client.get_token(
                username="{fill in super_user_username}",
                password="{fill in super_user_password}"
            )

            # Optional Step: For demonstration purpose, create a new user and disable them
            username = "{fill in username}"
            User.create(
                username=username,
                first_name="{fill in first_name}",
                last_name="{fill in last_name}",
                email="{fill in email}",
                password="{fill in password}",
                subscription_type="{fill in subscription_type}",
            )
            User.disable(user=username)

            # Display the details of the user you want to enable using their username/uuid
            print(User.details(username))

            # Enable the user
            # To enable multiple users, pass a list of username/uuid
            User.enable(user=username)

            # Check whether the user has been enabled
            # The disabled flag should be set to False, if the enable was sucessful
            print(User.details(username))

            ```
        """

        _users = user if isinstance(user, list) else [user]
        response_list = []
        for user in _users:
            user_uuid = User.details(user=user)["uuid"]
            url = f"/user/{user_uuid}/enable"
            response = Client._get_data(
                relative_url=check_and_append_otp_query_param(url, otp)
            )
            response_list.append(response)

        return pd.DataFrame(response_list)[User.USER_COLS]

    @staticmethod
    def details(user: Optional[str] = None) -> Dict[str, Union[str, bool]]:
        """Get user details

        Please do not pass the optional user parameter unless you are a super user. Only a
        super user can view details for other users.

        Args:
            user: Account user_uuid/username to get details. If not passed, then the currently logged-in
                details will be returned.

        Returns:
            A dict containing the details of the user

        Raises:
            ValueError: If the user_uuid/username is invalid or the user have insufficient permission to access other user's data
            ConnectionError: If the server address is invalid or not reachable.

        An example of displaying the logged-in user's uuid

        Example:
            ```python

            # Importing necessary libraries
            from  airt.client import User, Client

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Display the uuid of logged-in user
            # If you are a super user, you can view the details of other users by
            # passing their uuid/username in the user parameter.
            user_details = User.details()
            print(user_details["uuid"])

            # If a Non-super user tries to access other user's detail,
            # an exception will be thrown.
            try:
                User.details(user="some_other_username")
                print("Should not print this, the above line should raise an exception")
                raise RuntimeException()

            except ValueError as e:
                print("Expected to fail, everything is fine")
            ```
        """
        relative_url = (
            f"/user/details?user_uuid_or_name={user}"
            if user is not None
            else f"/user/details"
        )
        return Client._get_data(relative_url=relative_url)

    @staticmethod
    def update(
        user: Optional[str] = None,
        username: Optional[str] = None,
        first_name: Optional[str] = None,
        last_name: Optional[str] = None,
        email: Optional[str] = None,
        otp: Optional[str] = None,
    ) -> pd.DataFrame:
        """Update existing user details in the server.

         Please do not pass the optional user parameter unless you are a super user. Only a
         super user can update details for other users.

        Args:
            user: Account user_uuid/username to update. If not passed, then the default
                value **None** will be used to update the currently logged-in user details.
            username: New username for the user.
            first_name: New first name for the user.
            last_name: New last name for the user.
            email: New email for the user.
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
            A pandas DataFrame encapsulating the updated user details.

        Raises:
            ValueError: If the user_uuid/username is invalid or the user have insufficient permission to access other user's data
            ValueError: If the OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        Here's an example of updating the email address of the logged-in user

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import User, Client

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Display the registered email address of the logged-in user
            print(User.details()["email"])

            # Update the logged-in user's email address
            # If you are a super user, you can update the details of other users by
            # passing their uuid/username in the user parameter
            email = "{fill in new_email}"
            User.update(email=email)

            # Check whether the email address has been updated for the logged-in user
            print(User.details()["email"])

            # If you try to use an already used email address, an exception will be raised.
            try:
                User.update(email=email)
                print("Should not print this, the above line should raise an exception")
                raise RuntimeException()

            except ValueError as e:
                print("Expected to fail, everything is fine")
            ```
        """
        user_uuid = User.details(user=user)["uuid"]
        req_json = dict(
            username=username,
            first_name=first_name,
            last_name=last_name,
            email=email,
            otp=otp,
        )

        response = Client._post_data(
            relative_url=f"/user/{user_uuid}/update", json=req_json
        )

        return pd.DataFrame(response, index=[0])[User.USER_COLS]

    @staticmethod
    def _get_mfa_provision_url(otp: Optional[str] = None) -> str:
        """Get MFA provisioning url

        Args:
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
            The provisioning url
        """
        url = f"/user/mfa/generate"
        response = Client._get_data(
            relative_url=check_and_append_otp_query_param(url, otp)
        )
        return response["mfa_url"]

    @staticmethod
    def enable_mfa(otp: Optional[str] = None) -> PilImage:
        """Enable MFA for the user

        This method will generate a QR code. To finish the setup on your smartphone, scan the
        QR code with an authenticator app such as Google Authenticator and follow the on-device
        instructions. After you've completed the setup, call the `activate_mfa` method and pass the
        current OTP from the authenticator application to verify and enable MFA for your account

        Args:
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
            The generated QR code

        Here's an example to enable and activate MFA for the user

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import User, Client

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Check the current MFA status
            print(User.details()["is_mfa_active"])

            # Enable MFA for the user
            # The line below will generate a QR code. To finish the setup on your smartphone,
            # scan the QR code with an authenticator app like Google Authenticator and
            # follow the on-device instructions
            User.enable_mfa()

            # After you've completed the setup, enter the current OTP from the authenticator
            # app to verify and enable MFA for your account
            User.activate_mfa(otp="{fill in otp}")

            # Check the current MFA status
            # The is_mfa_active flag should be set to True, if the setup is successful
            print(User.details()["is_mfa_active"])
            ```
        """
        qr_code = qrcode.make(User._get_mfa_provision_url(otp))

        return qr_code

    @staticmethod
    def activate_mfa(otp: int) -> pd.DataFrame:
        """Activate MFA for the user

        Adding MFA to your account is a two-step process. To begin, you must enable MFA for your account
        by calling the `enable_mfa` method, then call the `activate_mfa` method and pass the current OTP
        from the authenticator application to verify and activate MFA for your account.

        Args:
            otp: Dynamically generated six-digit verification code from the authenticator app

        Returns:
            A pandas DataFrame encapsulating the MFA activated user details

        Raises:
            ValueError: If the user entered six-digit verification code is invalid

        Here's an example to enable and activate MFA for the user

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import User, Client

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Check the current MFA status
            print(User.details()["is_mfa_active"])

            # Enable MFA for the user
            # The line below will generate a QR code. To finish the setup on your smartphone,
            # scan the QR code with an authenticator app like Google Authenticator and
            # follow the on-device instructions
            User.enable_mfa()

            # After you've completed the setup, enter the current OTP from the authenticator
            # app to verify and enable MFA for your account
            User.activate_mfa(otp="{fill in otp}")

            # Check the current MFA status
            # The is_mfa_active flag should be set to True, if the setup is successful
            print(User.details()["is_mfa_active"])
            ```
        """
        response = Client._post_data(
            relative_url="/user/mfa/activate",
            json=dict(user_otp=otp),
        )

        return pd.DataFrame(response, index=[0])

    @staticmethod
    def disable_mfa(
        user: Optional[str] = None, otp: Optional[str] = None
    ) -> pd.DataFrame:
        """Disable MFA for the user

        We currently support two types of OTPs for disabling multi-factor authentication on your account.

        If you have access to the authenticator app, enter the app's dynamically generated six-digit verification
        code(OTP). If you don't have access to the authentication app, you can have an OTP sent to your registered
        phone number via SMS.

        To receive OTP via SMS, first call the `send_sms_otp` method, which will send the OTP to your registered
        phone number. Once you have the OTP, call the `disable_mfa` method to deactivate MFA for your account.

        Currently, we only support the above two methods for disabling MFA. If you do not have access to the authenticator
        app or your registered phone number, please contact your administrator.

        !!! note

            Please do not pass the user parameter unless you are a super user. Only
            a super user can disable MFA for other users.

        Args:
            user: Account user_uuid/username to disable MFA. If not passed, then the default
                value **None** will be used to disable MFA for the currently logged-in user.
            otp: Dynamically generated six-digit verification code from the authenticator app or
                the OTP you have received via SMS.

        Returns:
            A pandas DataFrame encapsulating the MFA disabled user details

        Raises:
            ValueError: If a non-super user tries to disable MFA for other users
            ValueError: If the OTP is invalid.
            ValueError: If the user_uuid/username is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        Here's an example of disabling MFA for the currently logged in user using the verification code generated by the authentication application.
        The example assumes that you have already activated the MFA on your account and have access to the authentication application.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Optional Step: Skip this step if you've already logged-in
            # Authenticate. Pass the current OTP from the authenticator app below
            otp="{fill in otp}"
            Client.get_token(
                username="{fill in username}",
                password="{fill in password}",
                otp=otp
            )

            # Check the current MFA status
            print(User.details()["is_mfa_active"])

            # Disable MFA for the user
            User.disable_mfa(otp=otp)

            # Check the current MFA status
            # The is_mfa_active flag should be set to False, if the disable was successful
            print(User.details()["is_mfa_active"])
            ```

        Here's an example of disabling MFA for the currently logged in user using the SMS OTP. The example assumes that you have
        already registered and validated your phone number on our servers.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Optional Step: Request OTP via SMS to authenticate
            # If you've already logged in, you can skip the two optional steps
            # If you've already have a valid token, skip the below optional step
            # and call Client.set_token instead of Client.get_token
            username="{fill in username}"
            User.send_sms_otp(
                username=username,
                message_template_name="get_token" # Don't change the message_template_name
            )

            # Optional Step: Authenticate using SMS OTP
            # The send_sms_otp method will send the OTP via SMS to the registered
            # phone number, which you must fill below
            password="{fill in password}"
            otp="{fill in otp}"
            Client.get_token(username=username, password=password, otp=otp)

            # Check the current MFA status
            print(User.details()["is_mfa_active"])

            # Request OTP via SMS to disable MFA
            User.send_sms_otp(
                username=username,
                message_template_name="disable_mfa" # Don't change the message_template_name
            )

            # The send_sms_otp method will send the OTP via SMS to the registered
            # phone number, which you must fill below
            User.disable_mfa(otp="{fill in otp}")

            # Check the current MFA status
            # The is_mfa_active flag should be set to False, if the disable was successful
            print(User.details()["is_mfa_active"])
            ```
        """

        user_uuid = User.details(user=user)["uuid"]
        url = f"/user/mfa/{user_uuid}/disable"
        response = Client._delete_data(
            relative_url=check_and_append_otp_query_param(url, otp)
        )

        return pd.DataFrame(response, index=[0])

    @staticmethod
    def enable_sso(sso_provider: str, sso_email: str, otp: Optional[str] = None) -> str:
        """Enable Single sign-on (SSO) for the user

        Args:
            sso_provider: Name of the Single sign-on (SSO) identity provider.
                At present, the API only supports **"google"** and **"github"** as valid SSO identity providers.
            sso_email: Email id to be used for SSO authentication
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
            A pandas DataFrame encapsulating the user details

        Here's an example of authenticating with Single sign-on (SSO) using google and
        setting the newly generated token to interact with the airt service.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import User, Client

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Enable single sign-on (SSO) and use google as the provider
            sso_provider="google"
            sso_email="{fill in sso_email}"
            User.enable_sso(sso_provider=sso_provider, sso_email=sso_email)

            # Authenticate using Single sign-on (SSO)
            # To generate a token using SSO, you must first authenticate with the provider.
            # The command below will generate an authorization URL for you.
            # Please copy and paste it into your preferred browser and complete the
            # SSO provider authentication within 10 minutes. Otherwise, the SSO login
            # will time out and you will need to call the get_token method again.
            sso_url = Client.get_token(sso_provider=sso_provider)
            print(sso_url)

            # Once the provider authentication is successful, call the below method to
            # set the generated token
            Client.set_sso_token()

            # If set_sso_token fails, the line below will throw an error.
            print(User.details())
            ```
        """
        response = Client._post_data(
            relative_url=f"/user/sso/enable",
            json=dict(sso_provider=sso_provider, sso_email=sso_email, otp=otp),
        )
        success_msg = f"Single sign-on (SSO) is successfully enabled for {sso_provider}. Please use {response['sso_email']} as the email address while authenticating with {sso_provider}."

        return success_msg

    @staticmethod
    def disable_sso(
        sso_provider: str,
        user: Optional[str] = None,
        otp: Optional[str] = None,
    ) -> str:
        """Disable Single sign-on (SSO) for the user

        Please do not pass the user parameter unless you are a super user. Only
        a super user can disable SSO for other users.

        Args:
            sso_provider: The name of the Single sign-on (SSO) provider you want to disable.
                At present, the API only supports **"google"** and **"github"** as valid SSO identity providers.
            user: Account user_uuid/username to disable SSO. If not passed, then the default
                value **None** will be used to disable SSO for the currently logged-in user.
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
            A pandas DataFrame encapsulating the SSO disabled user details

        Raises:
            ValueError: If a non-super user tries to disable SSO for other users
            ValueError: If the OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        Here's an example to disable the Single sign-on (SSO)

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import User, Client

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Optional Step: For demonstration purpose, enable Single sign-on (SSO)
            # for the user
            sso_provider="{fill in sso_provider}"
            sso_email="{fill in sso_email}"
            User.enable_sso(sso_provider=sso_provider, sso_email=sso_email)
            sso_url = Client.get_token(sso_provider=sso_provider) # Authenticate using SSO
            print(sso_url)
            Client.set_sso_token() # Set SSO token

            # Disable the Single sign-on (SSO) for the provider
            print(User.disable_sso(sso_provider=sso_provider))

            # If you try to disable an already disabled SSO provider, an exception
            # will be raised
            try:
                User.disable_sso(sso_provider=sso_provider)
                print("Should not print this, the above line should raise an exception")
                raise RuntimeException()

            except ValueError as e:
                print("Expected to fail, everything is fine")
            ```
        """

        user_uuid = User.details(user=user)["uuid"]
        url = f"/user/sso/{user_uuid}/disable/{sso_provider}"
        response = Client._delete_data(
            relative_url=check_and_append_otp_query_param(url, otp)
        )

        success_msg = f"Single sign-on (SSO) is successfully disabled for {response['sso_provider']}."

        return success_msg

    @staticmethod
    def register_phone_number(
        phone_number: Optional[str] = None,
        otp: Optional[str] = None,
    ) -> Dict[str, Union[str, bool]]:
        """Register a phone number

        Registering your phone number will help you to regain access in case you forget your password and cannot access
        your account. To receive the OTP via SMS, you need to register and validate your phone number. Calling this
        method will send an OTP via SMS to the phone number and you need to call the `validate_phone_number` method
        with the OTP you have received to complete the registration and validation process.

        Args:
            phone_number: Phone number to register. The phone number should follow the pattern of the country
                code followed by your phone number. For example, **440123456789, +440123456789,
                00440123456789, +44 0123456789, and (+44) 012 345 6789** are all valid formats for registering a
                UK phone number. If the phone number is not passed in the arguments, then the OTP will be sent to
                the phone number that was already registered to the user's account.
            otp: Dynamically generated six-digit verification code from the authenticator app. Please pass this
                parameter only if you have activated the MFA for your account.

        Returns:
            A dict containing the updated user details

        Raises:
            ValueError: If the OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        Here's an example of registering and validating a new phone number for the currently logged-in user

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Display the phone number that is currently registered
            # If no phone number has been registered, None will be displayed
            print(User.details()["phone_number"])

            # Register a new phone number.
            # If you only want to validate an existing phone number, call the
            # method below without the phone_number parameter
            User.register_phone_number(phone_number="{fill in phone_number}")

            # The above method will send the OTP via SMS to the specified phone number,
            # which you must enter below to complete the registration process
            User.validate_phone_number(otp="{fill in otp}")

            # Check whether the phone number has been updated and verified
            # The is_phone_number_verified flag should be set to True, if the
            # registration is successful
            user_details = User.details()
            print(user_details["phone_number"])
            print(user_details["is_phone_number_verified"])
            ```
        """
        if phone_number is not None:
            phone_number = standardize_phone_number(phone_number)

        req_json = dict(phone_number=phone_number, otp=otp)
        return Client._post_data(
            relative_url="/user/register_phone_number", json=req_json
        )

    @staticmethod
    def validate_phone_number(
        otp: Optional[str] = None,
    ) -> Dict[str, Union[str, bool]]:
        """Validate a registered phone number

        Please call the `register_phone_number` method to get the OTP via SMS and then call this method
        with the OTP to complete the registration and validation process.

        Args:
            otp: The OTP you have received on your registered phone number.

        Returns:
            A dict containing the updated user details

        Raises:
            ValueError: If the OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        Here's an example of registering and validating a new phone number for the currently logged-in user

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Authenticate
            Client.get_token(username="{fill in username}", password="{fill in password}")

            # Display the phone number that is currently registered
            # If no phone number has been registered, None will be displayed
            print(User.details()["phone_number"])

            # Register a new phone number.
            # If you only want to validate an existing phone number, call the
            # method below without the phone_number parameter
            User.register_phone_number(phone_number="{fill in phone_number}")

            # The above method will send the OTP via SMS to the specified phone number,
            # which you must enter below to complete the registration process
            User.validate_phone_number(otp="{fill in otp}")

            # Check whether the phone number has been updated and verified
            # The is_phone_number_verified flag should be set to True, if the
            # registration is successful
            user_details = User.details()
            print(user_details["phone_number"])
            print(user_details["is_phone_number_verified"])
            ```
        """
        url = "/user/validate_phone_number"

        return Client._get_data(relative_url=check_and_append_otp_query_param(url, otp))

    @staticmethod
    def send_sms_otp(username: str, message_template_name: str) -> str:
        """Send OTP via SMS to the user's registered phone number

        This method does not require a login, and you should only use it to reset your password,
        disable MFA, or generate a new token using SMS OTP.

        Calling this method will only send an OTP to your registered phone number via SMS. Following this method
        call, you should explicitly call `reset_password`, `disable_mfa`, or `Client.get_token` to complete
        the operation with the SMS OTP.

        Please remember to pass a valid message_template_name along with the request. At present, the API
        supports **"reset_password"**, **"disable_mfa"** and **"get_token"** as valid message templates.

        Args:
            username: Account username to send the OTP
            message_template_name: The message template to use while sending the OTP via SMS. At present,
                the API supports **"reset_password"**, **"disable_mfa"** and **"get_token"** as valid message templates

        Returns:
            The SMS status message

        Here's an example of a Non-MFA user resetting their password using the SMS OTP. The example assumes that you have
        already registered and validated your phone number on our servers.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Request OTP via SMS to the registered phone number
            # Please do not change the message_template_name
            username="{fill in username}"
            User.send_sms_otp(username=username, message_template_name="reset_password")

            # The above method will send the OTP via SMS to the registered phone number,
            # which you must fill below along with your new password
            new_password = "{fill in new_password}"
            otp = "{fill in otp}" # OTP received via SMS

            # Reset the password
            User.reset_password(username=username, new_password=new_password, otp=otp)

            # Authenticate using the new credentials
            # MFA users must pass the otp generated by the authenticator app below
            Client.get_token(username=username, password=new_password)

            # Check if get_token is successful
            print(User.details())
            ```

        Here's an example of how to disable MFA with SMS OTP, assuming you've already registered
        and validated your phone number on our servers but don't have a valid token.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Optional Step: Request OTP via SMS to authenticate
            # If you've already logged in, you can skip the two optional steps
            # If you've already have a valid token, skip the below optional step
            # and call Client.set_token instead of Client.get_token
            username="{fill in username}"
            User.send_sms_otp(
                username=username,
                message_template_name="get_token" # Don't change the message_template_name
            )

            # Optional Step: Authenticate using SMS OTP
            # The send_sms_otp method will send the OTP via SMS to the registered
            # phone number, which you must fill below
            password="{fill in password}"
            otp="{fill in otp}"
            Client.get_token(username=username, password=password, otp=otp)

            # Check the current MFA status
            print(User.details()["is_mfa_active"])

            # Request OTP via SMS to disable MFA
            User.send_sms_otp(
                username=username,
                message_template_name="disable_mfa" # Don't change the message_template_name
            )

            # The send_sms_otp method will send the OTP via SMS to the registered
            # phone number, which you must fill below
            User.disable_mfa(otp="{fill in otp}")

            # Check the current MFA status
            # The is_mfa_active flag should be set to False, if the disable was successful
            print(User.details()["is_mfa_active"])
            ```
        """
        url = f"/user/send_sms_otp?username={username}&message_template_name={message_template_name}"

        return Client._get_data(relative_url=url)

    @staticmethod
    def reset_password(username: str, new_password: str, otp: str) -> str:
        """Resets the password of an account either using a TOTP or SMS OTP.

        We currently support two types of OTPs to reset the password for your account and you don't have to be logged
        in to call this method

        If you have already activated the MFA for your account, then you can either pass the dynamically generated
        six-digit verification code from the authenticator app (TOTP) or you can also request an OTP via SMS to your registered phone number.

        If the MFA is not activated already, then you can only request the OTP via SMS to your registered phone number.

        To get OTP by SMS, you must first call `send_sms_otp` method which will send the OTP to your registered
        phone number. Once you have the OTP with you, then call this method with the OTP to reset your password.

        Currently, we only support the above two methods for resetting the password. In case, you don't have MFA enabled or don't
        have access to your registered phone number, please contact your administrator.

        Args:
            username: Account username to reset the password
            new_password: New password to set for the account
            otp: Dynamically generated six-digit verification code from the authenticator app or the OTP you have received
                via SMS.

        Returns:
            The password reset status message

        Raises:
            ValueError: If the username or OTP is invalid.
            ConnectionError: If the server address is invalid or not reachable.

        Here's an example for resetting the password using the verification code generated by the authentication application.
        The example assumes that you have already activated the MFA on your account and have access to the authentication application.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Details required to reset the password
            username = "{fill in username}"
            new_password = "{fill in new_password}"
            otp = "{fill in otp}" # OTP generated by the authenticator app

            # Reset the password
            User.reset_password(username=username, new_password=new_password, otp=otp)

            # Authenticate using the new credentials
            Client.get_token(username=username, password=new_password, otp=otp)

            # Check if get_token is successful
            print(User.details())
            ```

        Here's an example of a Non-MFA user resetting their password using the SMS OTP. The example assumes that you have already registered
        and validated your phone number on our servers.

        Example:
            ```python
            # Importing necessary libraries
            from  airt.client import Client, User

            # Request OTP via SMS to the registered phone number
            # Please do not change the message_template_name
            username="{fill in username}"
            User.send_sms_otp(username=username, message_template_name="reset_password")

            # The above method will send the OTP via SMS to the registered phone number,
            # which you must fill below along with your new password
            new_password = "{fill in new_password}"
            otp = "{fill in otp}" # OTP received via SMS

            # Reset the password
            User.reset_password(username=username, new_password=new_password, otp=otp)

            # Authenticate using the new credentials
            # MFA users must pass the otp generated by the authenticator app below
            Client.get_token(username=username, password=new_password)

            # Check if get_token is successful
            print(User.details())
            ```
        """
        req_json = dict(username=username, new_password=new_password, otp=otp)
        return Client._post_data(relative_url="/user/reset_password", json=req_json)  # type: ignore

In [ ]:
# Run example for User


run_examples_from_docstring(
    User,
    username=os.environ[SERVICE_USERNAME],
    password=os.environ[SERVICE_PASSWORD],
)

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import User, Client                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Display the uuid of logged-in user                                                                        │
│     user_details = User.details()                                                                               │
│     print(user_details["uuid"])                                                                                 │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ 7363f72b-7ebb-425c-832f-99c42fb7a016                                                                        │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── stderr ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for User.create

username = os.environ[SERVICE_SUPER_USER]
password = os.environ[SERVICE_PASSWORD]
username_for_new_test_user = f"random_user_{randrange(10000)}_{randrange(10000)}"
password_for_new_test_user = "example_password"

run_examples_from_docstring(
    User.create,
    super_user_username=username,
    super_user_password=password,
    username=username_for_new_test_user,
    first_name="example_first_name",
    last_name="example_last_name",
    email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
    password=password_for_new_test_user,
    super_user="False",
    subscription_type="test",
)

<module>:5: No type or annotation for parameter 'username'
<module>:6: No type or annotation for parameter 'first_name'
<module>:7: No type or annotation for parameter 'last_name'
<module>:8: No type or annotation for parameter 'email'
<module>:9: No type or annotation for parameter 'password'
<module>:10: No type or annotation for parameter 'subscription_type'
<module>:12: No type or annotation for parameter 'super_user'
<module>:14: No type or annotation for parameter 'phone_number'
<module>:17: No type or annotation for parameter 'otp'
<module>:21: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate with super user privileges                                                                   │
│     Client.get_token(                                                                                           │
│         username="{fill in super_user_username}",                                                               │
│         password="{fill in super_user_password}"                                                                │
│     )                                                                                                           │
│                                                                                                                 │
│     # Details required to create a new user                                                                     │
│     username = "{fill in username}"                                                                             │
│     first_name = "{fill in first_name}"                                                                         │
│     last_name = "{fill in last_name}"                                                                           │
│     email = "{fill in email}"                                                                                   │
│     password = "{fill in password}"                                                                             │
│     super_user = "{fill in super_user}"                                                                         │
│     subscription_type = "{fill in subscription_type}"                                                           │
│                                                                                                                 │
│     # Create a new user. To access this method, you must have super user privileges.                            │
│     new_user = User.create(                                                                                     │
│         username=username,                                                                                      │
│         first_name=first_name,                                                                                  │
│         last_name=last_name,                                                                                    │
│         email=email,                                                                                            │
│         password=password,                                                                                      │
│         super_user=super_user,                                                                                  │
│         subscription_type=subscription_type,                                                                    │
│     )                                                                                                           │
│                                                                                                                 │
│     # Display the details of the newly created user                                                             │
│     print(new_user)                                   

In [ ]:
# Run example for User.reset_password
# The below example is expected to throw an exception because
# the phone number is not registered for the test user.
# This is to avoid sending SMS each time we run the tests
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        User.reset_password,
        username=username_for_new_test_user,
        new_password=password_for_new_test_user,
        otp="123456",
        supress_stderr=True,
    )

<module>:17: No type or annotation for parameter 'username'
<module>:18: No type or annotation for parameter 'new_password'
<module>:19: No type or annotation for parameter 'otp'
<module>:23: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Details required to reset the password                                                                    │
│     username = "{fill in username}"                                                                             │
│     new_password = "{fill in new_password}"                                                                     │
│     otp = "{fill in otp}" # OTP generated by the authenticator app                                              │
│                                                                                                                 │
│     # Reset the password                                                                                        │
│     User.reset_password(username=username, new_password=new_password, otp=otp)                                  │
│                                                                                                                 │
│     # Authenticate using the new credentials                                                                    │
│     Client.get_token(username=username, password=new_password, otp=otp)                                         │
│                                                                                                                 │
│     # Check if get_token is successful                                                                          │
│     print(User.details())                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for User.send_sms_otp

# The below example is expected to throw an exception because
# the phone number is not registered for the test user
# This is to avoid sending SMS each time we run the tests


with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        User.send_sms_otp,
        username=username_for_new_test_user,
        new_password=username_for_new_test_user,
        password=username_for_new_test_user,
        otp="123456",
        supress_stderr=True,
    )

<module>:13: No type or annotation for parameter 'username'
<module>:14: No type or annotation for parameter 'message_template_name'
<module>:18: No type or annotation for returned value 1


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Request OTP via SMS to the registered phone number                                                        │
│     # Please do not change the message_template_name                                                            │
│     username="{fill in username}"                                                                               │
│     User.send_sms_otp(username=username, message_template_name="reset_password")                                │
│                                                                                                                 │
│     # The above method will send the OTP via SMS to the registered phone number,                                │
│     # which you must fill below along with your new password                                                    │
│     new_password = "{fill in new_password}"                                                                     │
│     otp = "{fill in otp}" # OTP received via SMS                                                                │
│                                                                                                                 │
│     # Reset the password                                                                                        │
│     User.reset_password(username=username, new_password=new_password, otp=otp)                                  │
│                                                                                                                 │
│     # Authenticate using the new credentials                                                                    │
│     # MFA users must pass the otp generated by the authenticator app below                                      │
│     Client.get_token(username=username, password=new_password)                                                  │
│                                                                                                                 │
│     # Check if get_token is successful                                                                          │
│     print(User.details())                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for User.disable_mfa
# The below example is expected to throw an exception because
# the default user is a NON-MFA activated user and when a NON-MFA user passes otp
# to get the token they will get this error
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        User.disable_mfa,
        username=username,
        password=password,
        otp="123456",
        supress_stderr=True,
    )

<module>:20: No type or annotation for parameter 'user'
<module>:22: No type or annotation for parameter 'otp'
<module>:26: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Optional Step: Skip this step if you've already logged-in                                                 │
│     # Authenticate. Pass the current OTP from the authenticator app below                                       │
│     otp="{fill in otp}"                                                                                         │
│     Client.get_token(                                                                                           │
│         username="{fill in username}",                                                                          │
│         password="{fill in password}",                                                                          │
│         otp=otp                                                                                                 │
│     )                                                                                                           │
│                                                                                                                 │
│     # Check the current MFA status                                                                              │
│     print(User.details()["is_mfa_active"])                                                                      │
│                                                                                                                 │
│     # Disable MFA for the user                                                                                  │
│     User.disable_mfa(otp=otp)                                                                                   │
│                                                                                                                 │
│     # Check the current MFA status                                                                              │
│     # The is_mfa_active flag should be set to False, if the disable was successful                              │
│     print(User.details()["is_mfa_active"])                                                                      │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for User.ls

username = os.environ[SERVICE_SUPER_USER]
password = os.environ[SERVICE_PASSWORD]

run_examples_from_docstring(
    User.ls,
    super_user_username=username,
    super_user_password=password,
)

<module>:5: No type or annotation for parameter 'offset'
<module>:6: No type or annotation for parameter 'limit'
<module>:7: No type or annotation for parameter 'disabled'
<module>:11: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate with super user privileges                                                                   │
│     Client.get_token(                                                                                           │
│         username="{fill in super_user_username}",                                                               │
│         password="{fill in super_user_password}"                                                                │
│     )                                                                                                           │
│                                                                                                                 │
│     # Display the details of all active users                                                                   │
│     # Set the disabled parameter to True to display the details of inactive users                               │
│     ux = User.ls()                                                                                              │
│     print(User.as_df(ux))                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                     uuid  ... is_phone_number_verified                                      │ │
│ │ 0   7363f72b-7ebb-425c-832f-99c42fb7a016  ...                    False                                      │ │
│ │ 1   50f41000-10e1-42a4-96ca-8d693d4b9fa1  ...                    False                                      │ │
│ │ 2   d882d2de-0a72-47df-9293-3a9f3563afea  ...                    False                                      │ │
│ │ 3   b418ccca-6e15-494d-bd82-4ba86d4b90f8  ...                    False                                      │ │
│ │ 4   c1c80aa6-d29c-43c1-8cd5-ac122ef20041  ...                    False                                      │ │
│ │ 5   2ec752ef-1cd1-4659-9272-7b6bab5083cb  ...                    False                                      │ │
│ │ 6   82435b53-fe09-4804-8c41-68850ecfb112  ...                    False                                      │ │
│ │ 7   ef0e86a2-2641-4c40-b0f5-2ae1af4634fe  ...                    False                                      │ │
│ │ 8   6a4cd131-9804-4d5a-8477-3a7124f353a6  ...                    False                                      │ │
│ │ 9   f577a97f-1107-43b3-bbd5-0bd4c73cf1c0  ...                    False                                      │ │
│ │ 10  dab20424-304f-4bcc-9e36-45b40364ae48  ...                    False                                      │ │
│ │ 11  449bee19-9078-4414-b0a8-141784aaee72  ...                    False                                      │ │
│ │ 12  e8229346-51b9-4a7b-aaaf-55946b470e0e  ...                    False                                      │ │
│ │ 13  6dc3e46b-d6da-41e0-a239-fbbc2e93fc30  ...                    False                                      │ │
│ │ 14  bf2dc2ee-960f-4fff-bcca-ba4402fd67bf  ...                    False                                      │ │
│ │                                                     

In [ ]:
# Run example for User.as_df

username = os.environ[SERVICE_SUPER_USER]
password = os.environ[SERVICE_PASSWORD]

run_examples_from_docstring(
    User.as_df,
    super_user_username=username,
    super_user_password=password,
)

<module>:3: No type or annotation for parameter 'ux'
<module>:6: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate with super user privileges                                                                   │
│     # Only super users can get the list of available users                                                      │
│     Client.get_token(                                                                                           │
│         username="{fill in super_user_username}",                                                               │
│         password="{fill in super_user_password}"                                                                │
│     )                                                                                                           │
│                                                                                                                 │
│     # Display the details of all active users                                                                   │
│     # Set the disabled parameter to True to display the details of inactive users                               │
│     ux = User.ls()                                                                                              │
│     print(User.as_df(ux))                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                     uuid  ... is_phone_number_verified                                      │ │
│ │ 0   7363f72b-7ebb-425c-832f-99c42fb7a016  ...                    False                                      │ │
│ │ 1   50f41000-10e1-42a4-96ca-8d693d4b9fa1  ...                    False                                      │ │
│ │ 2   d882d2de-0a72-47df-9293-3a9f3563afea  ...                    False                                      │ │
│ │ 3   b418ccca-6e15-494d-bd82-4ba86d4b90f8  ...                    False                                      │ │
│ │ 4   c1c80aa6-d29c-43c1-8cd5-ac122ef20041  ...                    False                                      │ │
│ │ 5   2ec752ef-1cd1-4659-9272-7b6bab5083cb  ...                    False                                      │ │
│ │ 6   82435b53-fe09-4804-8c41-68850ecfb112  ...                    False                                      │ │
│ │ 7   ef0e86a2-2641-4c40-b0f5-2ae1af4634fe  ...                    False                                      │ │
│ │ 8   6a4cd131-9804-4d5a-8477-3a7124f353a6  ...                    False                                      │ │
│ │ 9   f577a97f-1107-43b3-bbd5-0bd4c73cf1c0  ...                    False                                      │ │
│ │ 10  dab20424-304f-4bcc-9e36-45b40364ae48  ...                    False                                      │ │
│ │ 11  449bee19-9078-4414-b0a8-141784aaee72  ...                    False                                      │ │
│ │ 12  e8229346-51b9-4a7b-aaaf-55946b470e0e  ...                    False                                      │ │
│ │ 13  6dc3e46b-d6da-41e0-a239-fbbc2e93fc30  ...                    False                                      │ │
│ │ 14  bf2dc2ee-960f-4fff-bcca-ba4402fd67bf  ...       

In [ ]:
# Run example for User.enable_mfa
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        User.enable_mfa,
        username=username_for_new_test_user,
        password=password_for_new_test_user,
        otp="000000",
        supress_stderr=True,
    )

<module>:8: No type or annotation for parameter 'otp'
<module>:12: No type or annotation for returned value 1


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import User, Client                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Check the current MFA status                                                                              │
│     print(User.details()["is_mfa_active"])                                                                      │
│                                                                                                                 │
│     # Enable MFA for the user                                                                                   │
│     # The line below will generate a QR code. To finish the setup on your smartphone,                           │
│     # scan the QR code with an authenticator app like Google Authenticator and                                  │
│     # follow the on-device instructions                                                                         │
│     User.enable_mfa()                                                                                           │
│                                                                                                                 │
│     # After you've completed the setup, enter the current OTP from the authenticator                            │
│     # app to verify and enable MFA for your account                                                             │
│     User.activate_mfa(otp="{fill in otp}")                                                                      │
│                                                                                                                 │
│     # Check the current MFA status                                                                              │
│     # The is_mfa_active flag should be set to True, if the setup is successful                                  │
│     print(User.details()["is_mfa_active"])                                                                      │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ False                                                                                                       │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for User.activate_mfa
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        User.activate_mfa,
        username=username_for_new_test_user,
        password=password_for_new_test_user,
        otp="000000",
        supress_stderr=True,
    )

<module>:7: No type or annotation for parameter 'otp'
<module>:10: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import User, Client                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Check the current MFA status                                                                              │
│     print(User.details()["is_mfa_active"])                                                                      │
│                                                                                                                 │
│     # Enable MFA for the user                                                                                   │
│     # The line below will generate a QR code. To finish the setup on your smartphone,                           │
│     # scan the QR code with an authenticator app like Google Authenticator and                                  │
│     # follow the on-device instructions                                                                         │
│     User.enable_mfa()                                                                                           │
│                                                                                                                 │
│     # After you've completed the setup, enter the current OTP from the authenticator                            │
│     # app to verify and enable MFA for your account                                                             │
│     User.activate_mfa(otp="{fill in otp}")                                                                      │
│                                                                                                                 │
│     # Check the current MFA status                                                                              │
│     # The is_mfa_active flag should be set to True, if the setup is successful                                  │
│     print(User.details()["is_mfa_active"])                                                                      │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ False                                                                                                       │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
# Run example for User.enable_sso
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        User.enable_sso,
        username=username_for_new_test_user,
        password=password_for_new_test_user,
        sso_email="sso_email@mail.com",
        supress_stderr=True,
    )

<module>:3: No type or annotation for parameter 'sso_provider'
<module>:5: No type or annotation for parameter 'sso_email'
<module>:6: No type or annotation for parameter 'otp'
<module>:10: No type or annotation for returned value 1


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import User, Client                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Enable single sign-on (SSO) and use google as the provider                                                │
│     sso_provider="google"                                                                                       │
│     sso_email="{fill in sso_email}"                                                                             │
│     User.enable_sso(sso_provider=sso_provider, sso_email=sso_email)                                             │
│                                                                                                                 │
│     # Authenticate using Single sign-on (SSO)                                                                   │
│     # To generate a token using SSO, you must first authenticate with the provider.                             │
│     # The command below will generate an authorization URL for you.                                             │
│     # Please copy and paste it into your preferred browser and complete the                                     │
│     # SSO provider authentication within 10 minutes. Otherwise, the SSO login                                   │
│     # will time out and you will need to call the get_token method again.                                       │
│     sso_url = Client.get_token(sso_provider=sso_provider)                                                       │
│     print(sso_url)                                                                                              │
│                                                                                                                 │
│     # Once the provider authentication is successful, call the below method to                                  │
│     # set the generated token                                                                                   │
│     Client.set_sso_token()                                                                                      │
│                                                                                                                 │
│     # If set_sso_token fails, the line below will throw an error.                                               │
│     print(User.details())                                                                                       │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                     

In [ ]:
# Run example for User.disable_sso
with pytest.raises(RuntimeError) as e:
    run_examples_from_docstring(
        User.disable_sso,
        username=username_for_new_test_user,
        password=password_for_new_test_user,
        sso_provider="google",
        sso_email="sso_email@mail.com",
        supress_stderr=True,
    )

<module>:6: No type or annotation for parameter 'sso_provider'
<module>:8: No type or annotation for parameter 'user'
<module>:10: No type or annotation for parameter 'otp'
<module>:14: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import User, Client                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Optional Step: For demonstration purpose, enable Single sign-on (SSO)                                     │
│     # for the user                                                                                              │
│     sso_provider="{fill in sso_provider}"                                                                       │
│     sso_email="{fill in sso_email}"                                                                             │
│     User.enable_sso(sso_provider=sso_provider, sso_email=sso_email)                                             │
│     sso_url = Client.get_token(sso_provider=sso_provider) # Authenticate using SSO                              │
│     print(sso_url)                                                                                              │
│     Client.set_sso_token() # Set SSO token                                                                      │
│                                                                                                                 │
│     # Disable the Single sign-on (SSO) for the provider                                                         │
│     print(User.disable_sso(sso_provider=sso_provider))                                                          │
│                                                                                                                 │
│     # If you try to disable an already disabled SSO provider, an exception                                      │
│     # will be raised                                                                                            │
│     try:                                                                                                        │
│         User.disable_sso(sso_provider=sso_provider)                                                             │
│         print("Should not print this, the above line should raise an exception")                                │
│         raise RuntimeException()                                                                                │
│                                                                                                                 │
│     except ValueError as e:                                                                                     │
│         print("Expected to fail, everything is fine")                                                           │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr 

In [ ]:
# Run example for User.update

run_examples_from_docstring(
    User.update,
    username=username_for_new_test_user,
    password=password_for_new_test_user,
    new_email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
)

<module>:6: No type or annotation for parameter 'user'
<module>:8: No type or annotation for parameter 'username'
<module>:9: No type or annotation for parameter 'first_name'
<module>:10: No type or annotation for parameter 'last_name'
<module>:11: No type or annotation for parameter 'email'
<module>:12: No type or annotation for parameter 'otp'
<module>:16: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import User, Client                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Display the registered email address of the logged-in user                                                │
│     print(User.details()["email"])                                                                              │
│                                                                                                                 │
│     # Update the logged-in user's email address                                                                 │
│     # If you are a super user, you can update the details of other users by                                     │
│     # passing their uuid/username in the user parameter                                                         │
│     email = "{fill in new_email}"                                                                               │
│     User.update(email=email)                                                                                    │
│                                                                                                                 │
│     # Check whether the email address has been updated for the logged-in user                                   │
│     print(User.details()["email"])                                                                              │
│                                                                                                                 │
│     # If you try to use an already used email address, an exception will be raised.                             │
│     try:                                                                                                        │
│         User.update(email=email)                                                                                │
│         print("Should not print this, the above line should raise an exception")                                │
│         raise RuntimeException()                                                                                │
│                                                                                                                 │
│     except ValueError as e:                                                                                     │
│         print("Expected to fail, everything is fine")                                                           │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ random_user_4408_9299@email.com                                                                             │ │
│ │ random_user_6228_1855@email.com                                                                             │ │
│ │ Expected to fail, everything is fine                                                                        │ │
│ │                                                     

In [ ]:
# Run example for User.details

run_examples_from_docstring(
    User.details,
    username=username_for_new_test_user,
    password=password_for_new_test_user,
)

<module>:6: No type or annotation for parameter 'user'
<module>:10: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import User, Client                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Display the uuid of logged-in user                                                                        │
│     # If you are a super user, you can view the details of other users by                                       │
│     # passing their uuid/username in the user parameter.                                                        │
│     user_details = User.details()                                                                               │
│     print(user_details["uuid"])                                                                                 │
│                                                                                                                 │
│     # If a Non-super user tries to access other user's detail,                                                  │
│     # an exception will be thrown.                                                                              │
│     try:                                                                                                        │
│         User.details(user="some_other_username")                                                                │
│         print("Should not print this, the above line should raise an exception")                                │
│         raise RuntimeException()                                                                                │
│                                                                                                                 │
│     except ValueError as e:                                                                                     │
│         print("Expected to fail, everything is fine")                                                           │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │ bf2dc2ee-960f-4fff-bcca-ba4402fd67bf                                                                        │ │
│ │ Expected to fail, everything is fine                                                                        │ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭────────────────────────────────────────────────── stderr ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰───────────────────────────────────────────────────────

In [ ]:
# Run example for User.register_phone_number
# The below example will throw Incorrect username or password, thats expected because
# if we send valid username and password, the sms will be sent to the passed phone number.
# We don't want to send the SMS each time we run the tests
with pytest.raises(RuntimeError) as e:

    invalid_phone_number = "910000000000"
    invalid_otp = "000000"
    run_examples_from_docstring(
        User.register_phone_number,
        username="invalid_username",
        password="invalid_username",
        phone_number=invalid_phone_number,
        otp=invalid_otp,
        supress_stderr=True,
    )

<module>:8: No type or annotation for parameter 'phone_number'
<module>:13: No type or annotation for parameter 'otp'
<module>:17: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Display the phone number that is currently registered                                                     │
│     # If no phone number has been registered, None will be displayed                                            │
│     print(User.details()["phone_number"])                                                                       │
│                                                                                                                 │
│     # Register a new phone number.                                                                              │
│     # If you only want to validate an existing phone number, call the                                           │
│     # method below without the phone_number parameter                                                           │
│     User.register_phone_number(phone_number="{fill in phone_number}")                                           │
│                                                                                                                 │
│     # The above method will send the OTP via SMS to the specified phone number,                                 │
│     # which you must enter below to complete the registration process                                           │
│     User.validate_phone_number(otp="{fill in otp}")                                                             │
│                                                                                                                 │
│     # Check whether the phone number has been updated and verified                                              │
│     # The is_phone_number_verified flag should be set to True, if the                                           │
│     # registration is successful                                                                                │
│     user_details = User.details()                                                                               │
│     print(user_details["phone_number"])                                                                         │
│     print(user_details["is_phone_number_verified"])                                                             │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────

In [ ]:
# Run example for User.validate_phone_number
# The below example will throw Incorrect username or password, thats expected because
# if we send valid username and password, the OTP will be verified with the messaging service
# We don't want to send an external call each time we run the tests
with pytest.raises(RuntimeError) as e:

    invalid_phone_number = "910000000000"
    invalid_otp = "000000"
    run_examples_from_docstring(
        User.validate_phone_number,
        username="invalid_username",
        password="invalid_username",
        phone_number=invalid_phone_number,
        otp=invalid_otp,
        supress_stderr=True,
    )

<module>:6: No type or annotation for parameter 'otp'
<module>:9: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate                                                                                              │
│     Client.get_token(username="{fill in username}", password="{fill in password}")                              │
│                                                                                                                 │
│     # Display the phone number that is currently registered                                                     │
│     # If no phone number has been registered, None will be displayed                                            │
│     print(User.details()["phone_number"])                                                                       │
│                                                                                                                 │
│     # Register a new phone number.                                                                              │
│     # If you only want to validate an existing phone number, call the                                           │
│     # method below without the phone_number parameter                                                           │
│     User.register_phone_number(phone_number="{fill in phone_number}")                                           │
│                                                                                                                 │
│     # The above method will send the OTP via SMS to the specified phone number,                                 │
│     # which you must enter below to complete the registration process                                           │
│     User.validate_phone_number(otp="{fill in otp}")                                                             │
│                                                                                                                 │
│     # Check whether the phone number has been updated and verified                                              │
│     # The is_phone_number_verified flag should be set to True, if the                                           │
│     # registration is successful                                                                                │
│     user_details = User.details()                                                                               │
│     print(user_details["phone_number"])                                                                         │
│     print(user_details["is_phone_number_verified"])                                                             │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── stdout ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭───────────────────────────────────────────── stderr supressed ──────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ ╰─────────────────────────────────────────────────────

In [ ]:
# Run example for User.disable

username = os.environ[SERVICE_SUPER_USER]
password = os.environ[SERVICE_PASSWORD]
username_for_new_test_user = f"random_user_{randrange(10000)}_{randrange(10000)}"
password_for_new_test_user = "example_password"

run_examples_from_docstring(
    User.disable,
    super_user_username=username,
    super_user_password=password,
    username=username_for_new_test_user,
    first_name="example_first_name",
    last_name="example_last_name",
    email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
    password=password_for_new_test_user,
    subscription_type="test",
)

<module>:5: No type or annotation for parameter 'user'
<module>:6: No type or annotation for parameter 'otp'
<module>:10: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate with super user privileges                                                                   │
│     Client.get_token(                                                                                           │
│         username="{fill in super_user_username}",                                                               │
│         password="{fill in super_user_password}"                                                                │
│     )                                                                                                           │
│                                                                                                                 │
│     # Optional Step: For demonstration purpose, create a new user                                               │
│     username = "{fill in username}"                                                                             │
│     User.create(                                                                                                │
│         username=username,                                                                                      │
│         first_name="{fill in first_name}",                                                                      │
│         last_name="{fill in last_name}",                                                                        │
│         email="{fill in email}",                                                                                │
│         password="{fill in password}",                                                                          │
│         subscription_type="{fill in subscription_type}",                                                        │
│     )                                                                                                           │
│                                                                                                                 │
│     # Display the details of the user you want to disable using their username/uuid                             │
│     print(User.details(username))                                                                               │
│                                                                                                                 │
│     # Disable the user                                                                                          │
│     # To disable multiple users, pass a list of username/uuid                                                   │
│     User.disable(user=username)                                                                                 │
│                                                                                                                 │
│     # Check whether the user has been disabled                                                                  │
│     # The disabled flag should be set to True, if the disable was sucessful                                     │
│     print(User.details(username))                                                                               │
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── st

In [ ]:
# Run example for User.enable

username = os.environ[SERVICE_SUPER_USER]
password = os.environ[SERVICE_PASSWORD]
username_for_new_test_user = f"random_user_{randrange(10000)}_{randrange(10000)}"
password_for_new_test_user = "example_password"

run_examples_from_docstring(
    User.enable,
    super_user_username=username,
    super_user_password=password,
    username=username_for_new_test_user,
    first_name="example_first_name",
    last_name="example_last_name",
    email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
    password=password_for_new_test_user,
    subscription_type="test",
)

<module>:5: No type or annotation for parameter 'user'
<module>:6: No type or annotation for parameter 'otp'
<module>:10: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Example:                                                                                                        │
│                                                                                                                 │
│     # Importing necessary libraries                                                                             │
│     from  airt.client import Client, User                                                                       │
│                                                                                                                 │
│     # Authenticate with super user privileges                                                                   │
│     Client.get_token(                                                                                           │
│         username="{fill in super_user_username}",                                                               │
│         password="{fill in super_user_password}"                                                                │
│     )                                                                                                           │
│                                                                                                                 │
│     # Optional Step: For demonstration purpose, create a new user and disable them                              │
│     username = "{fill in username}"                                                                             │
│     User.create(                                                                                                │
│         username=username,                                                                                      │
│         first_name="{fill in first_name}",                                                                      │
│         last_name="{fill in last_name}",                                                                        │
│         email="{fill in email}",                                                                                │
│         password="{fill in password}",                                                                          │
│         subscription_type="{fill in subscription_type}",                                                        │
│     )                                                                                                           │
│     User.disable(user=username)                                                                                 │
│                                                                                                                 │
│     # Display the details of the user you want to enable using their username/uuid                              │
│     print(User.details(username))                                                                               │
│                                                                                                                 │
│     # Enable the user                                                                                           │
│     # To enable multiple users, pass a list of username/uuid                                                    │
│     User.enable(user=username)                                                                                  │
│                                                                                                                 │
│     # Check whether the user has been enabled                                                                   │
│     # The disabled flag should be set to False, if the enable was sucessful                                     │
│     print(User.details(username))                                                                               │
│                                                       

In [ ]:
# A helper context manager for logging as super user and accessing user routes.


@contextmanager
def super_user():

    # 1. Get token
    username = os.environ[SERVICE_SUPER_USER]
    password = os.environ[SERVICE_PASSWORD]

    Client.get_token(username=username, password=password)

    yield


with super_user():

    server, auth_token = Client._get_server_url_and_token()

    display(f"{len(server)=}")
    display(f"{len(auth_token)=}")

    assert len(server) > 0
    assert len(auth_token) > 0

'len(server)=24'

'len(auth_token)=127'

In [ ]:
# tests for reset_password
username = "invalid_username"
new_password = "new_password"
invalid_otp = "123456"
with pytest.raises(ValueError) as e:
    User.reset_password(username, new_password, invalid_otp)

assert "Something went wrong" in str(e.value), str(e.value)
display(str(e.value))

'Something went wrong. The username or OTP you entered is incorrect. Please try again or contact your administrator.'

In [ ]:
# tests for send_sms_otp
username = "invalid_username"
message_template_name = "message_template_name"
actual = User.send_sms_otp(username, message_template_name)
display(actual)
assert "If you have already registered and verified your phone number" in actual

'If you have already registered and verified your phone number, you will receive the OTP by SMS. If you did not receive the OTP, please contact your administrator.'

In [ ]:
# Tests for User.details

# Positive scenarios: Super user trying to access their user details

with super_user():
    # Positive scenario: Super user trying to access their user details
    actual = User.details()
    assert len(actual["uuid"].replace("-", "")) == 32

    actual = User.details(user=actual["username"])
    assert len(actual["uuid"].replace("-", "")) == 32

    actual = User.details(user=actual["uuid"])
    assert len(actual["uuid"].replace("-", "")) == 32

    # Positive scenario: Super user trying to access other user details
    other_user = User.ls()[0]
    actual = User.details(user=other_user.uuid)
    assert actual["uuid"] == other_user.uuid

    # Negative scenario: Super user trying to access invalid user details
    random_user_name = "random_user_name"
    with pytest.raises(ValueError) as e:
        User.details(user=random_user_name)
    display(str(e.value))

    with pytest.raises(ValueError) as e:
        User.details(user=INVALID_UUID_FOR_TESTING)
    display(str(e.value))

'Incorrect username. Please try again.'

'The user uuid is incorrect. Please try again.'

In [ ]:
# Tests for User.details

# Authenticating as normal user
username = os.environ[SERVICE_USERNAME]
password = os.environ[SERVICE_PASSWORD]
Client.get_token(username=username, password=password)

# Positive scenario: Normal user trying to access their user details
actual = User.details()
assert len(actual["uuid"].replace("-", "")) == 32

actual = User.details(user=actual["username"])
assert len(actual["uuid"].replace("-", "")) == 32

actual = User.details(user=actual["uuid"])
assert len(actual["uuid"].replace("-", "")) == 32
actual["uuid"]

# Negative scenarios: Normal user trying to access other user details
random_user_name = "random_user_name"
with pytest.raises(ValueError) as e:
    User.details(user=random_user_name)
assert str(e.value) == "Insufficient permission to access other user's data"
display(str(e.value))


with pytest.raises(ValueError) as e:
    User.details(user=INVALID_UUID_FOR_TESTING)
assert str(e.value) == "Insufficient permission to access other user's data"
display(str(e.value))

"Insufficient permission to access other user's data"

"Insufficient permission to access other user's data"

In [ ]:
# Testing negative scenario. Non-super user trying to create a new user
with pytest.raises(ValueError) as e:
    User.create(
        username=f"random_user_{randrange(10000)}_{randrange(10000)}",
        first_name="example_first_name",
        last_name="example_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password="example_password",
        super_user=False,
        subscription_type="test",
    )

display(str(e.value))

'You do not have sufficient permission to access this route. Please contact your administrator for help.'

In [ ]:
# tests for User.create

# Positive Scenario. Create a new user

with super_user():

    for ph in ["+44123456789", "0044123456789", "44123456789"]:
        _user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
        _email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"

        response_df = User.create(
            username=_user_name,
            first_name="random_first_name",
            last_name="random_last_name",
            email=_email,
            password="random_password",
            super_user=False,
            phone_number=ph,
            subscription_type="test",
        )

        display(response_df[["uuid", "created", "phone_number"]])
        assert response_df["phone_number"][0] == "44123456789"

        assert response_df.shape == (1, len(User.USER_COLS))

        # Trying to create new user with existing email and username

        with pytest.raises(ValueError) as e:
            User.create(
                username=_user_name,
                first_name="random_first_name",
                last_name="random_last_name",
                email=_email,
                password="random_password",
                super_user=False,
                subscription_type="test",
            )

        display(str(e.value))

,uuid,created,phone_number
0,9147b0a7-f8e4-46db-9935-7f90434c99ab,2022-10-27T08:41:54,44123456789


'The requested username or email already exists. Try another.'

,uuid,created,phone_number
0,74b06ad9-5eea-4276-8ba1-47d39464d6aa,2022-10-27T08:41:54,44123456789


'The requested username or email already exists. Try another.'

,uuid,created,phone_number
0,dead5d28-db6f-435c-b2fa-79cc62f19ac9,2022-10-27T08:41:55,44123456789


'The requested username or email already exists. Try another.'

In [ ]:
# tests for User.create

# Negative Scenario. Passing wrong subscription_type

with super_user():

    with pytest.raises(ValueError) as e:
        User.create(
            username=f"random_user_{randrange(10000)}_{randrange(10000)}",
            first_name="random_first_name",
            last_name="random_last_name",
            email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
            password="random_password",
            super_user=False,
            subscription_type="fake-value",
        )

    display(f"{str(e.value)=}")
    assert "value is not a valid enumeration member" in str(e.value)

    # Negative Scenario. Non-MFA user passing otp

    random_otp = 123456

    with pytest.raises(ValueError) as e:
        User.create(
            username=f"random_user_{randrange(10000)}_{randrange(10000)}",
            first_name="random_first_name",
            last_name="random_last_name",
            email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
            password="random_password",
            super_user=False,
            subscription_type="small",
            otp=random_otp,
        )

    display(f"{str(e.value)=}")
    assert "MFA is not activated for the account" in str(e.value)

'str(e.value)=\'[{\\\'loc\\\': [\\\'body\\\', \\\'subscription_type\\\'], \\\'msg\\\': "value is not a valid enumeration member; permitted: \\\'small\\\', \\\'medium\\\', \\\'large\\\', \\\'infobip\\\', \\\'captn\\\', \\\'test\\\', \\\'superuser\\\'", \\\'type\\\': \\\'type_error.enum\\\', \\\'ctx\\\': {\\\'enum_values\\\': [\\\'small\\\', \\\'medium\\\', \\\'large\\\', \\\'infobip\\\', \\\'captn\\\', \\\'test\\\', \\\'superuser\\\']}}]\''

"str(e.value)='MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'"

In [ ]:
# Tests for User ls

with super_user():

    ux = User.ls()

    display(f"{len(ux)=}")
    assert len(ux) > 0

    # Testing list with offset and limit
    offset = 1
    limit = 3

    ux = User.ls(offset=offset, limit=limit)

    display(f"{len(ux)=}")
    assert 0 < len(ux) <= limit

    # Testing list with invalid offset and limit
    offset = 1_000_000_000
    limit = 3

    ux = User.ls(offset=offset, limit=limit)

    display(f"{len(ux)=}")
    assert ux == []

    # Testing with disabled flag

    all_ux = User.ls()
    disabled_ux = User.ls(disabled=True)

    display(f"{len(disabled_ux)=}")
    assert len(disabled_ux) >= 0
    assert len(all_ux) > len(disabled_ux)

'len(ux)=19'

'len(ux)=3'

'len(ux)=0'

'len(disabled_ux)=1'

In [ ]:
# Tests for User.as_df:

ux = User.ls()

df = User.as_df(ux)

assert df.shape == (len(ux), len(User.USER_COLS))

df[["uuid", "created"]]

,uuid,created
0,7363f72b-7ebb-425c-832f-99c42fb7a016,2022-10-27T04:39:16
1,50f41000-10e1-42a4-96ca-8d693d4b9fa1,2022-10-27T04:39:16
2,d882d2de-0a72-47df-9293-3a9f3563afea,2022-10-27T04:39:16
3,b418ccca-6e15-494d-bd82-4ba86d4b90f8,2022-10-27T08:22:26
4,c1c80aa6-d29c-43c1-8cd5-ac122ef20041,2022-10-27T08:22:27
5,2ec752ef-1cd1-4659-9272-7b6bab5083cb,2022-10-27T08:22:28
6,82435b53-fe09-4804-8c41-68850ecfb112,2022-10-27T08:22:29
7,ef0e86a2-2641-4c40-b0f5-2ae1af4634fe,2022-10-27T08:22:30
8,6a4cd131-9804-4d5a-8477-3a7124f353a6,2022-10-27T08:22:31
9,f577a97f-1107-43b3-bbd5-0bd4c73cf1c0,2022-10-27T08:22:32


In [ ]:
# Tests for User.as_df:
# Passing empty ux list

ux = []

df = User.as_df(ux)

assert df.shape == (len(ux), len(User.USER_COLS))

df[["uuid", "created"]]

,uuid,created


In [ ]:
# tests for user.disable

with super_user():

    # create a new user

    response_df = User.create(
        username=f"random_user_{randrange(10000)}_{randrange(10000)}",
        first_name="random_first_name",
        last_name="random_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password="random_password",
        super_user=False,
        subscription_type="test",
    )

    display(response_df[["uuid", "created"]])
    assert not response_df["disabled"][0]

    # Disable the newly created user

    disable_response = User.disable(response_df["username"][0])

    display(disable_response[["uuid", "created"]])
    assert disable_response["disabled"][0]

    # Disable already disabled user

    with pytest.raises(ValueError) as e:
        User.disable(response_df["uuid"][0])

    display(str(e.value))

    # Negative Scenario. Non-MFA user passing otp

    with pytest.raises(ValueError) as e:
        random_otp = 123456
        User.disable(response_df["uuid"][0], otp=random_otp)

    display(str(e.value))

,uuid,created
0,e2e3cf00-2368-406c-9dad-d16159e1c562,2022-10-27T08:41:56


,uuid,created
0,e2e3cf00-2368-406c-9dad-d16159e1c562,2022-10-27T08:41:56


'The user has already been disabled.'

'MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'

In [ ]:
with super_user():

    # create a new user

    user_1 = User.create(
        username=f"random_user_{randrange(10000)}_{randrange(10000)}",
        first_name="random_first_name",
        last_name="random_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password="random_password",
        super_user=False,
        subscription_type="test",
    )

    display(user_1[["uuid", "created"]])
    assert not user_1["disabled"][0]

    # create a new user

    user_2 = User.create(
        username=f"random_user_{randrange(10000)}_{randrange(10000)}",
        first_name="random_first_name",
        last_name="random_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password="random_password",
        super_user=False,
        subscription_type="test",
    )

    display(user_2[["uuid", "created"]])
    assert not user_2["disabled"][0]

    # Disable the newly created user

    disable_response = User.disable([user_1["username"][0], user_2["uuid"][0]])

    display(disable_response[["uuid", "created"]])
    assert disable_response["disabled"][0]

    assert disable_response["disabled"].all()
    assert disable_response.shape[0] == 2
    disable_response

,uuid,created
0,9b15b7d2-1737-4c9d-8285-a4477d60d820,2022-10-27T08:41:57


,uuid,created
0,80cad6e8-d6aa-4753-936f-d04b56d890ab,2022-10-27T08:41:57


,uuid,created
0,9b15b7d2-1737-4c9d-8285-a4477d60d820,2022-10-27T08:41:57
1,80cad6e8-d6aa-4753-936f-d04b56d890ab,2022-10-27T08:41:57


In [ ]:
# tests for user.enable

with super_user():

    # create a new user

    response_df = User.create(
        username=f"random_user_{randrange(10000)}_{randrange(10000)}",
        first_name="random_first_name",
        last_name="random_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password="random_password",
        super_user=False,
        subscription_type="test",
    )

    # Disable the newly created user
    disable_response = User.disable(response_df["uuid"][0])

    display(disable_response[["uuid", "created"]])
    assert disable_response["disabled"][0]

    # Enable the disabled user

    enable_response = User.enable(disable_response["username"][0])

    display(enable_response[["uuid", "created"]])
    assert not enable_response["disabled"][0]

    # Enable already enabled user

    with pytest.raises(ValueError) as e:
        User.enable(enable_response["uuid"][0])

    display(str(e.value))

,uuid,created
0,9b6d5893-fd2b-46a0-aca8-c0575d2899a9,2022-10-27T08:41:58


,uuid,created
0,9b6d5893-fd2b-46a0-aca8-c0575d2899a9,2022-10-27T08:41:58


'The user has already been enabled.'

In [ ]:
# tests for user.enable. Enabling multiple users

with super_user():

    # create a new user

    response_df_user_1 = User.create(
        username=f"random_user_{randrange(10000)}_{randrange(10000)}",
        first_name="random_first_name",
        last_name="random_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password="random_password",
        super_user=False,
        subscription_type="test",
    )

    # Disable the newly created user
    disable_response_user_1 = User.disable(response_df_user_1["uuid"][0])
    display(disable_response_user_1[["uuid", "created"]])
    assert disable_response_user_1["disabled"][0]

    # create a new user

    response_df_user_2 = User.create(
        username=f"random_user_{randrange(10000)}_{randrange(10000)}",
        first_name="random_first_name",
        last_name="random_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password="random_password",
        super_user=False,
        subscription_type="test",
    )

    # Disable the newly created user
    disable_response_user_2 = User.disable(response_df_user_2["uuid"][0])
    display(disable_response_user_2[["uuid", "created"]])
    assert disable_response_user_2["disabled"][0]

    # Enable the disabled users
    enable_response = User.enable(
        [disable_response_user_1["username"][0], disable_response_user_2["uuid"][0]]
    )

    display(enable_response[["uuid", "created"]])
    assert not enable_response["disabled"].all()
    assert enable_response.shape[0] == 2
    enable_response
    # Negative Scenario. Non-MFA user passing otp
    with pytest.raises(ValueError) as e:
        random_otp = 123456
        User.enable(
            [
                disable_response_user_1["username"][0],
                disable_response_user_2["uuid"][0],
            ],
            otp=random_otp,
        )

    display(str(e.value))

,uuid,created
0,a2746326-0a5d-4aec-ad5b-30aaf3ed264f,2022-10-27T08:41:58


,uuid,created
0,651f8d55-359b-4347-8422-62b7c03f0b0d,2022-10-27T08:41:59


,uuid,created
0,a2746326-0a5d-4aec-ad5b-30aaf3ed264f,2022-10-27T08:41:58
1,651f8d55-359b-4347-8422-62b7c03f0b0d,2022-10-27T08:41:59


'MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'

In [ ]:
# tests for User.update

# Testing positive scenario. Calling User.update as a super user

with super_user():

    # Creating a new user

    _username = f"random_user_{randrange(10000)}_{randrange(10000)}"

    response_df = User.create(
        username=_username,
        first_name="random_first_name",
        last_name="random_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password="random_password",
        super_user=False,
        subscription_type="test",
    )

    display(response_df[["uuid", "created"]])

    assert response_df.shape == (1, len(User.USER_COLS))

    # updating the user details as super user
    _new_username = f"random_user_{randrange(10000)}_{randrange(10000)}"
    df = User.update(user=response_df["uuid"][0], username=_new_username)
    assert df["username"][0] == _new_username

    _new_lastname = f"random_user_{randrange(10000)}_{randrange(10000)}"
    df = User.update(user=_new_username, last_name=_new_lastname)
    display(df[["uuid", "created"]])
    assert df["last_name"][0] == _new_lastname

,uuid,created
0,e0f7e48d-9a2f-44a1-94db-f22c9fa47932,2022-10-27T08:41:59


,uuid,created
0,e0f7e48d-9a2f-44a1-94db-f22c9fa47932,2022-10-27T08:41:59


In [ ]:
# tests for User.update

# Testing positive scenario. Calling User.update as a normal user

new_user_details: dict = {}

with super_user():
    global new_user_details

    # Creating a new user

    _username = f"random_user_{randrange(10000)}_{randrange(10000)}"
    _password = "password"

    response_df = User.create(
        username=_username,
        first_name="random_first_name",
        last_name="random_last_name",
        email=f"random_user_{randrange(10000)}_{randrange(10000)}@email.com",
        password=_password,
        super_user=False,
        subscription_type="test",
    )

    display(response_df[["uuid", "created"]])

    new_user_details.update(
        user=response_df["username"][0], username=_username, password=_password
    )

    assert response_df.shape == (1, len(User.USER_COLS))


# Authenticating with newely created user credentials
Client.get_token(
    username=new_user_details["username"], password=new_user_details["password"]
)

# updating the user details
_new_username = f"random_user_{randrange(10000)}_{randrange(10000)}"
_new_first_name = "random_new_first_name"
_new_last_name = "random_last_first_name"
_new_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"

response_df = User.update(
    username=_new_username,
    first_name=_new_first_name,
    last_name=_new_last_name,
    email=_new_email,
)

display(response_df[["uuid", "created"]])
assert response_df.shape == (1, len(User.USER_COLS))
assert response_df["username"][0] == _new_username
assert response_df["first_name"][0] == _new_first_name
assert response_df["last_name"][0] == _new_last_name
assert response_df["email"][0] == _new_email


# Authenticating with newly created password
Client.get_token(username=_new_username, password=_password)

display(f"Masked token = { '*' * (len(Client.auth_token))}")
assert len(Client.auth_token) >= 127

# Negative scenario: Non-super user trying to update other user details
with pytest.raises(ValueError) as e:

    User.update(
        user=INVALID_UUID_FOR_TESTING,
        username=_new_username,
        first_name=_new_first_name,
        last_name=_new_last_name,
        email=_new_email,
    )
display(str(e.value))

# Negative Scenario. Non-MFA user passing otp
with pytest.raises(ValueError) as e:
    random_otp = 123456
    User.update(
        username=_new_username,
        first_name=_new_first_name,
        last_name=_new_last_name,
        email=_new_email,
        otp=random_otp,
    )
display(str(e.value))
assert "MFA is not activated for the account." in str(e.value)

,uuid,created
0,9118e747-8948-405e-990c-37a7aaf92525,2022-10-27T08:42:00


,uuid,created
0,9118e747-8948-405e-990c-37a7aaf92525,2022-10-27T08:42:00


'Masked token = *************************************************************************************************************************************************'

"Insufficient permission to access other user's data"

'MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'

In [ ]:
# Tests for _get_mfa_provision_url

mfa_url = User._get_mfa_provision_url()
display(f"mfa_url: {'*'*len(mfa_url)}")
assert len(mfa_url) > 0

'mfa_url: ***************************************************************************************************************'

In [ ]:
# Tests for enable_mfa

qr_code = User.enable_mfa()

assert type(qr_code) == PilImage
type(qr_code)

# Negative Scenario. Non-MFA user passing otp
with pytest.raises(ValueError) as e:
    random_otp = 123456
    User.enable_mfa(otp=random_otp)
display(str(e.value))
assert "MFA is not activated for the account." in str(e.value)

'MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'

In [ ]:
# Tests for activate_mfa. Negative case

random_otp = 123123
with pytest.raises(ValueError) as e:
    User.activate_mfa(otp=random_otp)
str(e.value)

'Invalid OTP. Please try again.'

In [ ]:
# Tests for disable_mfa. Negative case: Non-MFA enabled user trying to disable MFA for self
with pytest.raises(ValueError) as e:
    User.disable_mfa()

assert str(e.value) == "MFA is already disabled for the user", str(e.value)

# Tests for disable_mfa. Negative case: Normal user trying to disable MFA for other users
with pytest.raises(ValueError) as e:
    User.disable_mfa(user=INVALID_UUID_FOR_TESTING)

assert str(e.value) == "Insufficient permission to access other user's data", str(
    e.value
)
display("OK")

# Negative Scenario. Non-MFA user passing otp
with pytest.raises(ValueError) as e:
    random_otp = 123456
    User.disable_mfa(otp=random_otp)
display(str(e.value))
assert "MFA is not activated for the account." in str(e.value)

'OK'

'MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'

In [ ]:
# A helper context manager to create new users for testing
@contextmanager
def create_normal_user_for_testing():
    # 1. Get Super user token
    username = os.environ[SERVICE_SUPER_USER]
    password = os.environ[SERVICE_PASSWORD]

    Client.get_token(username=username, password=password)

    _user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
    _email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"
    _password = "random_password"

    response_df = User.create(
        username=_user_name,
        first_name="random_first_name",
        last_name="random_last_name",
        email=_email,
        password=_password,
        super_user=False,
        subscription_type="test",
    )

    #     display(response_df[["id", "created"]])

    assert response_df.shape == (1, len(User.USER_COLS))

    Client.get_token(username=_user_name, password=_password)

    yield

In [ ]:
# Tests for disable_sso. Negative case: Non-SSO user trying to disable SSO for self
with create_normal_user_for_testing():

    sso_provider = "google"

    with pytest.raises(ValueError) as e:
        User.disable_sso(sso_provider=sso_provider)

    assert "SSO is not enabled" in str(e.value)

    # Negative case: Normal user trying to disable SSO for other users
    with pytest.raises(ValueError) as e:
        User.disable_sso(sso_provider=sso_provider, user=INVALID_UUID_FOR_TESTING)

    assert "Insufficient permission" in str(e.value), str(e.value)
    display("OK")

'OK'

In [ ]:
# Tests for enable_sso
with create_normal_user_for_testing():

    # Positive case: Non-MFA user enabling SSO
    sso_email = "sso_email@mail.com"
    sso_provider = "google"
    actual = User.enable_sso(sso_provider, sso_email)
    display(actual)
    assert sso_email in actual
    assert sso_provider in actual
    # Non-MFA user disabling SSO
    actual = User.disable_sso(sso_provider=sso_provider)
    display(actual)
    assert sso_provider in actual

    # Positive case: Non-MFA user enabling SSO with new email address
    new_sso_email = "new_sso_email@mail.com"
    sso_provider = "google"
    actual = User.enable_sso(sso_provider, new_sso_email)
    display(actual)
    assert new_sso_email in actual
    assert sso_provider in actual

    # Negative case: Non-MFA user enabling SSO by passing OTP
    with pytest.raises(ValueError) as e:
        random_otp = 123456
        User.enable_sso(sso_provider, sso_email, otp=random_otp)
    display(str(e.value))
    assert "MFA is not activated for the account." in str(e.value)

'Single sign-on (SSO) is successfully enabled for google. Please use sso_email@mail.com as the email address while authenticating with google.'

'Single sign-on (SSO) is successfully disabled for google.'

'Single sign-on (SSO) is successfully enabled for google. Please use new_sso_email@mail.com as the email address while authenticating with google.'

'MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'

In [ ]:
# Tests for register_phone_number
with create_normal_user_for_testing():
    # Negative Scenario. Non-MFA user passing otp
    with pytest.raises(ValueError) as e:
        random_otp = 123456
        User.disable_mfa(otp=random_otp)
    display(str(e.value))
    assert "MFA is not activated for the account." in str(e.value)

'MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.'

In [ ]:
# Tests for validate_phone_number
with create_normal_user_for_testing():
    # Negative Scenario. Calling validate_phone_number without registering a phone number
    with pytest.raises(ValueError) as e:
        random_otp = 123456
        User.validate_phone_number(otp=random_otp)
    display(str(e.value))
    assert "The phone number is not yet registered" in str(e.value)

'The phone number is not yet registered. Please register your phone number before calling this method.'